# Поиск качественных спеллчекеров 

Задача: проверить несколько спеллчекеров для поиска максимально удобного и качественного, создать функцию их очистки.

# Метрика

Использовать в качестве метрики будем так:

1 - wer(чистый_текст, текст_спеллчекера)/ wer_тестового_текста_от_чистого

Метрика будет стремиться к 1.

In [1]:
def wer(reference, hypothesis):
    # Разбиваем строки на слова, разделяя пробелами
    ref_words = reference.split()
    hyp_words = hypothesis.split()
    
    # Создаем матрицу расстояний для вычисления расстояния Левенштейна
    matrix = [[0] * (len(hyp_words) + 1) for _ in range(len(ref_words) + 1)]
    
    # Инициализируем первую строку и первый столбец матрицы
    for i in range(len(ref_words) + 1):
        matrix[i][0] = i
    for j in range(len(hyp_words) + 1):
        matrix[0][j] = j
    
    # Заполняем матрицу расстояний
    for i in range(1, len(ref_words) + 1):
        for j in range(1, len(hyp_words) + 1):
            cost = 0 if ref_words[i - 1] == hyp_words[j - 1] else 1
            matrix[i][j] = min(matrix[i - 1][j] + 1, matrix[i][j - 1] + 1, matrix[i - 1][j - 1] + cost)
    
    # Расстояние Левенштейна между эталоном и распознанной последовательностями
    distance = matrix[len(ref_words)][len(hyp_words)]
    
    # Вычисляем WER
    wer = float(distance) / len(ref_words)
    
    return round(wer, 4)

Также мы будем отслеживать скорость работы алгоритмов.

In [2]:
import time

def measure_time(func, *args, **kwargs):
    start_time = time.time()
    func(*args, **kwargs)
    end_time = time.time()
    return round(end_time - start_time, 4)

# Тестирование

Мы подготовили два текста для тестирования спеллчекеров: один на русском, другой на английском.

In [3]:
text1_train = '''Привет, клиенты! Мы вас преветствуем в нашем телефонном оператаре - МобилТел. 
У нас сейчас шикарние акциии на новии смартфони и тарифние плони. Если вы скорее заключете договор, вы можите получити смартфони по половиной цене. 
Это замечательно шанс сэкономить на своем новом устройстве. Также, у нас есть неограничены тарифи с началной ставкой всего в 500 рублей в месяц. 
Не упустите эту фантастическу офферту! Звоните нам прямо сейчас, чтобы пообщатся с одним из наших дружелюбних менеджеров. 
Благодарим вас за выбор МобилТел
Сант-Петербург
Санкт Петербург
питер
ирина и виктор'''

In [4]:
text1_check = '''Привет, клиенты! Мы вас приветствуем в нашем телефонном операторе - МобилТел. 
У нас сейчас шикарные акции на новые смартфоны и тарифные планы. Если вы скорее заключите договор, вы можете получить смартфоны по половине цены. 
Это замечательный шанс сэкономить на своем новом устройстве. Также, у нас есть неограниченные тарифы с начальной ставкой всего в 500 рублей в месяц. 
Не упустите эту фантастическую оферту! Звоните нам прямо сейчас, чтобы пообщаться с одним из наших дружелюбных менеджеров. 
Благодарим вас за выбор МобилТел.
Санкт-Петербург
Санкт-Петербург
Питер
Ирина и Виктор
'''

In [5]:
text2_train = '''Hello, custumer suppor here. I'm writhing to inform yuo about our lates promossions on phonnes and daa plans. 
Our mobil operator, Cellnet, has exsciting deels that you won't want to miss. If yuo act now, yuo can get a brand new Iphone for halff the price. 
It's a great oppurtunity to save big on your new smartfone. Also, we have unlimitid data planns startin at just $20 per month. 
Don't mis out on this amasing ofer. Call us todai to speek with one of our frendly represntatives. 
Thank yuo for chosing Cellnet!'''

In [6]:
text2_check = '''Hello, customer support here. I'm writing to inform you about our latest promotions on phones and data plans. 
Our mobile operator, Cellnet, has exciting deals that you won't want to miss. If you act now, you can get a brand new iPhone for half the price. 
It's a great opportunity to save big on your new smartphone. Also, we have unlimited data plans starting at just $20 per month. 
Don't miss out on this amazing offer. Call us today to speak with one of our friendly representatives. 
Thank you for choosing Cellnet!'''

In [7]:
wer1 = wer(text1_check, text1_train)
wer1

0.3412

In [8]:
wer2 = wer(text2_check, text2_train)
wer2

0.3152

In [9]:
results = [["spellchecker", "ru_text", "ru_time", "en_text", "en_time", "multilang"]]

# Яндекс

In [10]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [11]:
import requests
import json
from tqdm import tqdm

s = "Мама \nмыла мылам \nрамму "
url = f"https://speller.yandex.net/services/spellservice.json/checkText?text={s}"

# Выполняем GET-запрос
response = requests.get(url)

# Проверка статуса ответа
if response.status_code == 200:
    # Разбор JSON-ответа
    data = json.loads(response.text)
    
    # Ваш код обработки данных здесь
    
    print(data)
else:
    print('Ошибка при запросе:', response.status_code)

[{'code': 1, 'pos': 11, 'row': 1, 'col': 5, 'len': 5, 'word': 'мылам', 's': ['мылом', 'мыла']}, {'code': 1, 'pos': 18, 'row': 2, 'col': 0, 'len': 5, 'word': 'рамму', 's': ['раму', 'рамку', 'маму']}]


In [12]:
def clean(text, data):
    if data != []:
        for i in data:
            text = text.replace(i["word"], i["s"][0])
        return text
    else:
        return text

In [13]:
def req(s, return_word=False):
    url = f"https://speller.yandex.net/services/spellservice.json/checkText?text={s}"
    response = requests.get(url)
    if response.status_code == 200:
        data = json.loads(response.text)
        if return_word == False:
            return clean(s, data)
        elif return_word == True:
            if len(data)>0:
                return clean(s, data), [[s, i["word"], i["s"][0]] for i in data]
            else:
                return clean(s, data), "NM"

In [14]:
print(req("Здеся нет коттов"))

Здесь нет котов


In [15]:
req("Здеся нет коттов", return_word=True)

('Здесь нет котов',
 [['Здеся нет коттов', 'Здеся', 'Здесь'],
  ['Здеся нет коттов', 'коттов', 'котов']])

In [16]:
results.append(["Yandex", 
                round(1 - wer(text1_check, req(text1_train))/wer1, 4), #0.55
                measure_time(req, text1_train),                        #0.196
                round(1 - wer(text2_check, req(text2_train))/wer2, 4), #0.90
                measure_time(req, text2_train),                        #0.3431
                "yes"
               ])

# language-tool

In [17]:
pip install language_tool_python

Note: you may need to restart the kernel to use updated packages.


In [18]:
import language_tool_python
tool_ru = language_tool_python.LanguageToolPublicAPI('ru')
tool_en = language_tool_python.LanguageToolPublicAPI('en_BR')

In [19]:
text = 'Здеся нет коттов'
tool_ru.check(text)
tool_ru.correct(text)

'Здесь нет котлов'

In [20]:
matches = tool_ru.check(text)

In [21]:
matches[0].replacements[0]

'Здесь'

In [22]:
tool = language_tool_python.LanguageToolPublicAPI('ru')
def langt(sen, no_matches = True, tool = tool_ru):
    if no_matches:
        return tool.correct(sen)
    else:
        matches = tool.check(sen)
        return tool.correct(sen), [match.replacements[0] for match in matches]

In [23]:
langt('He is an cat', tool = tool_en)

'He is a cat'

In [24]:
langt('Здеся нет коттов')

'Здесь нет котлов'

In [25]:
langt('Здеся нет коттов', no_matches=False)

('Здесь нет котлов', ['Здесь', 'котлов'])

In [26]:
results.append(["language-tool", 
                round(1 - wer(text1_check, langt(text1_train))/wer1, 4),                 #0.3104
                measure_time(langt, text1_train),                                        #0.5109
                round(1 - wer(text2_check, langt(text2_train, tool = tool_en))/wer2, 4), #0.0
                measure_time(langt, text2_train),                                        #0.5452
                "no"
               ])

# pyspellchecker

In [27]:
pip install pyspellchecker

Note: you may need to restart the kernel to use updated packages.


In [28]:
from spellchecker import SpellChecker
import re

spell_en = SpellChecker()
spell_ru = SpellChecker(language='ru')

# find those words that may be misspelled
misspelled = spell_en.unknown(['something', 'is', 'hapenning', 'here'])

for word in misspelled:
    # Get the one `most likely` answer
    print(spell_en.correction(word))

    # Get a list of `likely` options
    print(spell_en.candidates(word))

happening
{'happening', 'henning', 'penning'}


In [29]:
misspelled

{'hapenning'}

In [30]:
%%time
spell_en.unknown(['something', ',', 'hapenning', 'here'])

CPU times: total: 0 ns
Wall time: 0 ns


{'hapenning'}

In [31]:
text = "Hello, Ron!"

In [32]:
tokens = re.findall(r'\w+|\W+', text)
tokens

['Hello', ', ', 'Ron', '!']

In [33]:
import re
def spell(text, spell=spell_ru, misspelled_return = False):
    clean_text = []
    if misspelled_return:
        bad = []
    for line in text.split("\n"):
        sentences = [i.replace(' ', '') for i in re.findall(r'\w+|\W+', text) if i != " "]
        corrected_words = []
        for word in sentences:
            if len(word)>1:
                corrected_word = spell.correction(word)
                if corrected_word == None:
                    corrected_word = word
            else:
                corrected_word = word.replace(" ", "")
            corrected_words.append(corrected_word)
        misspelled = spell.unknown(sentences)
        clean_text.append(corrected_words)
        if misspelled_return:
            bad.append(misspelled)
    if misspelled_return:
        clean_text = '\n'.join([' '.join(line) for line in clean_text if isinstance(line, list)])
        return re.sub(r'\s+([!?,-.])', r'\1', clean_text), bad
    else:
        clean_text = '\n'.join([' '.join(line) for line in clean_text if isinstance(line, list)])
        return re.sub(r'\s+([!?,-.])', r'\1', clean_text)

In [ ]:
import re
from spellchecker import SpellChecker

def spell(text, spell=SpellChecker(language='ru'), misspelled_return=False):
    clean_text = []
    bad = [] if misspelled_return else None

    for line in text.split("\n"):
        sentences = [i.replace(' ', '') for i in re.findall(r'\w+|\W+', line) if i != " "]
        corrected_words = [spell.correction(word) if len(word) > 1 else word.replace(" ", "") for word in sentences]
        misspelled = spell.unknown(sentences)

        clean_text.append(corrected_words)

        if misspelled_return:
            bad.append(misspelled)

    clean_text = '\n'.join([' '.join(line) for line in clean_text if isinstance(line, list)])

    return (re.sub(r'\s+([!?,-.])', r'\1', clean_text), bad) if misspelled_return else re.sub(r'\s+([!?,-.])', r'\1', clean_text)

In [34]:
spell("The neext, ttext", spell_en)

'The next, text'

In [35]:
spell('Здеся нет коттов')

'здесь нет готов'

In [36]:
spell("МобилТел")

'МобилТел'

In [37]:
results.append(["pyspellchecker", 
                round(1 - wer(text1_check, spell(text1_train))/wer1, 4),                 #-25.0671
                measure_time(spell, text1_train),                                        #49.4834
                round(1 - wer(text2_check, spell(text2_train, spell = spell_en))/wer2, 4), #-14.5869
                measure_time(spell, text2_train),                                        #41.4297
                "no"
               ])

# Тестирование

In [38]:
import pandas as pd

In [39]:
pd.DataFrame(results[1:], columns=results[0])

,spellchecker,ru_text,ru_time,en_text,en_time,multilang
0,Yandex,0.5519,0.5088,0.8966,0.2615,yes
1,language-tool,0.3104,1.1699,0.0000,55.4561,no
2,pyspellchecker,-25.0671,116.7624,-14.5869,99.0206,no


# Выводы

Самым качественным, удобным и быстрым оказался спеллчекер от Яндекса.